## Dataset description
In the figure of the experimental setup (see CWRU website), motor is at the left. Faulty bearings with known fault depth (0.007 inch, or 0.014 inch, or 0.021 inch) are installed in the motor and the resulting vibration signal is collected by accelerometers. Accelerometers are placed at three locations of the motor: at the drive end (DE), fan end (FE) and base (BA). The end of the motor connected to the rotor shaft is called drive end. The other end is called fan end. It is called fan end because almost every induction motor has a fan at that side. The purpose of the fan is to cool the motor. Base is the support over which the motor is mounted. So accelerometers are placed at these three locations and resulting time series data are collected with a sampling frequency of either 12k or 48k. Sampling frequency is nothing but how many data points we collect in a second. So a sampling frequency of 48k means that the sensor will collect 48000 data points per second. If our data file contains 4800000 data points for example, then that would mean we have collected data for 100 seconds at a sampling frequency of 48k.

In the experiment, different amounts of (torsional) loads are applied to the motor. In all of our experiments we have taken data with 1hp load and 48k sampling frequency. Data files are originally stored in `.mat` format. `.mat` is the default format of data storage in `MATLAB`. `MATLAB` is heavily used in engineering disciplines. So several public mechanical datasets are stored in `.mat` format. It just makes it easy to load the data into `MATLAB` and do further analysis. In python, we can read `.mat` files using `scipy` library.

In [1]:
import glob
from scipy.io import loadmat
import numpy as np

To reiterate, in our analysis we have used data with 1hp load and 48k sampling frequency. We can download the files from the website and put those in one folder. In our case, the folder is `/home/biswajit/data/cwru/`. In the website, the data are stored using numbers as file names. For example, Ball defect of 0.007 inch depth and 1hp load is stored as `123.mat`. Similarly for other files. We have appended the fault type, its depth and load information to each file name after downloading for convenience. For Normal file, I have appended "Time" to the left. I have done so to make it the last entry of the sorted list. 

In [2]:
files = glob.glob("/home/biswajit/data/cwru/*")
files = np.sort(files)
files

array(['/home/biswajit/data/cwru/B007_1_123.mat',
       '/home/biswajit/data/cwru/B014_1_190.mat',
       '/home/biswajit/data/cwru/B021_1_227.mat',
       '/home/biswajit/data/cwru/IR007_1_110.mat',
       '/home/biswajit/data/cwru/IR014_1_175.mat',
       '/home/biswajit/data/cwru/IR021_1_214.mat',
       '/home/biswajit/data/cwru/OR007_6_1_136.mat',
       '/home/biswajit/data/cwru/OR014_6_1_202.mat',
       '/home/biswajit/data/cwru/OR021_6_1_239.mat',
       '/home/biswajit/data/cwru/Time_Normal_1_098.mat'], dtype='<U46')

While reading each mat files using `loadmat` function of `scipy`, the result is a dictionary. Its keys give us the data.

In [3]:
check_data = loadmat(files[0])
type(check_data)

dict

In [4]:
check_data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'X123_DE_time', 'X123_FE_time', 'X123RPM'])

Keys of each file contain drive end signal (DE_time), fan end signal (FE_time), and in some cases base signal (BA). `X123RPM` gives RPM information. We will only take drive end data for our analysis. Drive ends are closer to the bearing. So we believe, it would give more reliable indication of fault. Thus, we have used only drive end data. We can get drive end data using its key.

In [5]:
drive_end_data = check_data["X123_DE_time"]
drive_end_data.shape

(487384, 1)

Of all the files in the folder, one file (`IR014_1_175.mat`) is rather peculiar. It has following keys: `'X217'`, `'X217_DE_time'`, `'X175_DE_time'`, `'X175_FE_time'`, `'X175RPM'`. `'X175_DE_time'` contains less number of data points than we need for further analysis. So we have taken `'X217_DE_time'` for this particular file.

In [6]:
check_data_2 = loadmat(files[4])
check_data_2.keys()

dict_keys(['__header__', '__version__', '__globals__', 'X217', 'X217_DE_time', 'X175_DE_time', 'X175_FE_time', 'X175RPM'])

## Data preprocessing
Before the deep learning era, it was quite difficult to analyze raw data to get results. Therefore, people used classical feature based techniques for fault diagnosis. In feature based techniques, first relevant features are collected from the data. Features can be categorized as time domain features, frequency domain features or time-frequency domain features.

Time domain features are maximum value of signal, minimum value of the signal, root mean square (RMS) value, or statistical features such as skewness, kurtosis, etc. These are called time domain features because the features are computed from time domain data.

Sometimes, it is more beneficial to transform data to frequency domain (by doing FFT) and then collect features from frequency domain signal. In that case, those features are called frequency domain features. These features are usually, the frequency amplitude values at rotational frequency and its multiples.

Yet another useful transformation of data is time-frequency transformation. A spectrogram gives us time-frequency representation. Another time-frequency transformation is obtained by Wavelet transform. After wavelet transform, we can collect features like "wavelet packet energy" and "wavelet packet entropy". These are the types of features we have used heavily in our experiments. It turns out that these features mostly give good results for vibration signal. But the downside is that it is computationally expensive to compute these features.

Now, to compute time domain features, we can either take the full signal or segments of it. If we take full signal and compute, say, 8 features, it will give us 8 numbers. These 8 numbers are too less to represent more than 480000 data points. A better alternative is to segment the original data into segments of length 1024 or 2048 without any overlap between consecutive segments. This will give us more segments that would hopefully represent the original data in a better way.

So first, let's compute the maximum number of segments of length 1024 that we can extract from each file. We will use only drive end data.

### Number of segments from each file

In [7]:
for file in files:
    data = loadmat(file)
    if file[-7:-4] == "175":         # Peculiar IR014_1 data
        key = "X217_DE_time"
    else:
        key = "X" + file[-7:-4] + "_DE_time"
    drive_end_data = data[key]
    num_segments = np.floor(len(drive_end_data)/1024)
    print(num_segments)

475.0
474.0
475.0
474.0
477.0
473.0
475.0
473.0
477.0
472.0


So we can extract at least 470 segments from each file. In the processed data, I have taken first 460 segments from each file for no good reason. After extraction, our data will be of size $(4600 \times 1024)$. Then if we want to collect features, we can collect features taking each row of this data. For example, if we collect 8 time domain features for each segment, our feature matrix will have a shape of $(4600 \times 8)$. Here, we will not compute features. Rather, we will just create a preprocessed data matrix.

In [8]:
segmented_data = np.repeat(np.nan, 4600*1024).reshape(4600,1024)
num = 0
for file in files:
    data = loadmat(file)
    if file[-7:-4] == "175":    
        key = "X217_DE_time"
    else:
        key = "X" + file[-7:-4] + "_DE_time"
    drive_end_data = data[key]
    for i in range(460):           # We collect 460 segments from each file
        segmented_data[num,:] = drive_end_data[i*1024:(i+1)*1024, 0]
        num = num + 1

Check whether `segmented_data` contains any `nan` values or not.

In [9]:
np.sum(np.isnan(segmented_data))

0

For comparison, first download the processed data available online at [this link](https://github.com/biswajitsahoo1111/cbm_codes_open/blob/master/notebooks/data/CWRU_48k_load_1_CNN_data.npz). Load the data and then compare it with previously prepared "segmented_data".

In [10]:
read_online_data = np.load("/home/biswajit/data/CWRU_48k_load_1_CNN_data.npz")
read_online_data.files

['data', 'labels']

In [11]:
online_data = read_online_data["data"]
online_data.shape

(4600, 32, 32)

In [12]:
online_data = online_data.reshape(4600, -1)
online_data.shape

(4600, 1024)

Now we will check whether both files contain same data or not.

In [13]:
4600*1024

4710400

In [14]:
np.sum(np.isclose(online_data, segmented_data, atol = 1e-4))

4710400

So the values in both files are nearly same with an absolute tolerance of $10^{-4}$. We can also check a few individual values to convince ourselves.

In [15]:
online_data[:5,:5]

array([[-0.041097 , -0.046104 , -0.028372 ,  0.0012517,  0.033796 ],
       [ 0.048399 ,  0.048399 ,  0.050902 ,  0.050485 ,  0.040889 ],
       [ 0.18671  ,  0.31876  ,  0.39846  ,  0.41076  ,  0.36988  ],
       [-0.17378  , -0.10598  , -0.030041 ,  0.042975 ,  0.12705  ],
       [-0.0039637, -0.031501 , -0.057369 , -0.071764 , -0.064879 ]])

In [16]:
segmented_data[:5,:5]

array([[-0.04109723, -0.046104  , -0.02837169,  0.00125169,  0.03379569],
       [ 0.04839877,  0.04839877,  0.05090215,  0.05048492,  0.04088862],
       [ 0.18671077,  0.31876431,  0.39845538,  0.41076369,  0.36987508],
       [-0.17377662, -0.10597662, -0.03004062,  0.04297477,  0.12704677],
       [-0.00396369, -0.03150092, -0.05736923, -0.07176369, -0.06487938]])

The online data has few significant digits. This is because it was prepared first by extracting data from `mat` files and saving it in `csv` files. Those `csv` files were then used to load data and ultimately save it in `npz` format. 

Now if we want to compute features, we can compute it using segmented data. Steps to [compute time domain features](https://github.com/biswajitsahoo1111/cbm_codes_open/blob/master/notebooks/Calculating_time_domain_features_CWRU.ipynb) are explained in [this notebook](https://github.com/biswajitsahoo1111/cbm_codes_open/blob/master/notebooks/Calculating_time_domain_features_CWRU.ipynb). In my [project page](https://biswajitsahoo1111.github.io/cbm_codes_open/), I have also mentioned ways to compute wavelet packet features ([wavelet packet energy](https://github.com/biswajitsahoo1111/cbm_codes_open/blob/master/notebooks/calculate_wavelet_packet_energy_features.ipynb) as well as [wavelet packet entropy](https://github.com/biswajitsahoo1111/cbm_codes_open/blob/master/notebooks/calculate_wavelet_packet_entropy_features.ipynb)). We can also apply deep learning techniques to the segmented time domain data without computing any features what so ever. [This notebook](https://github.com/biswajitsahoo1111/cbm_codes_open/blob/master/notebooks/Deep_learning_based_fault_diagnosis_using_CNN_on_raw_time_domain_data.ipynb) applies convolutional neural network (CNN) to time domain data of the present notebook and achieves **98.7%** overall accuracy.